Build a sentiment analysis / polarity model

Sentiment analysis can be casted as a binary text classification problem,
that is fitting a linear classifier on features extracted from the text
of the user messages so as to guess whether the opinion of the author is
positive or negative.

# Topics

In [1]:
#!pip install sklearn
#!pip install pandas
#!pip install numpy
#!pip install scikit-learn
#!pip install scipy
#!pip install nltk
#!pip install random2
#!pip install warnings-plugin

import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
from pandas import DataFrame, read_csv
import pandas as pd
import sklearn.preprocessing
from scipy import stats
from sklearn.decomposition import NMF
import random
import nltk
#import nltk.downloads('stopwords')
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)



In [2]:
dataset = pd.read_csv("/Users/nicoweiner/Dropbox/Studium Amsterdam/Q3 Emerging Technology/Agradi Report/Scraper/Agradi_reviews.csv", sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
pd.options.display.max_columns = None

In [3]:
#importing the stopwords

stopWords = set (stopwords.words('dutch'))

print(len(stopWords))
print(stopWords)

101
{'wordt', 'in', 'om', 'kunnen', 'uit', 'daar', 'hun', 'meer', 'was', 'ze', 'moet', 'haar', 'hoe', 'ben', 'kan', 'iemand', 'niets', 'veel', 'heeft', 'alles', 'al', 'hier', 'heb', 'er', 'wil', 'hem', 'zijn', 'maar', 'want', 'naar', 'hij', 'toen', 'nu', 'geen', 'dus', 'ook', 'of', 'deze', 'doch', 'uw', 'andere', 'dit', 'mij', 'zelf', 'het', 'geweest', 'wie', 'niet', 'wezen', 'hebben', 'worden', 'zo', 'werd', 'voor', 'zonder', 'van', 'dat', 'als', 'ge', 'omdat', 'je', 'op', 'zich', 'na', 'wat', 'eens', 'een', 'die', 'men', 'aan', 'zou', 'dan', 'onder', 'ons', 'u', 'toch', 'iets', 'bij', 'mijn', 'doen', 'met', 'altijd', 'is', 'had', 'zij', 'door', 'me', 'reeds', 'waren', 'tot', 'zal', 'der', 'te', 'ja', 'de', 'ik', 'nog', 'over', 'en', 'tegen', 'kon'}


In [4]:
#create a document term matrix with TFIDF and cleaning the data of stopwords

tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words=stopWords, token_pattern='[a-zA-Z][a-zA-Z]{2,}')
doc_term_matrix = tfidf_vect.fit_transform(dataset['Review'].values.astype('U'))

In [5]:
#create a probability matrix, given topics is set to 25

nmf = NMF(n_components=15, random_state=42)
nmf.fit(doc_term_matrix)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=15, random_state=42, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [6]:
#retrieve the probability vector of words 

first_topic = nmf.components_[0]
top_topic_words = first_topic.argsort()[-10:]

In [7]:
#the words in the data grouped by topics

for i,topic in enumerate(nmf.components_):
    print(f'Top words for topic #{i}:')
    print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top words for topic #0:
['uur', 'website', 'niemand', 'chaps', 'even', 'goede', 'super', 'levering', 'service', 'snelle']


Top words for topic #1:
['besteld', 'binnen', 'gaat', 'opnieuw', 'gemaild', 'foto', 'maand', 'wanneer', 'twee', 'deken']


Top words for topic #2:
['graafwerk', 'gestuurd', 'gevraagd', 'geven', 'geval', 'gewoon', 'betaalbaar', 'leuke', 'geweldige', 'site']


Top words for topic #3:
['bestellen', 'niks', 'gewoon', 'retourneren', 'moeilijk', 'steeds', 'contact', 'oplichting', 'ruilen', 'slecht']


Top words for topic #4:
['elders', 'duurde', 'vervangend', 'voordat', 'waardeloos', 'ingediend', 'maand', 'bedrijf', 'geleverd', 'artikel']


Top words for topic #5:
['excusses', 'vertraging', 'geplaatst', 'gezegd', 'januari', 'december', 'werkdagen', 'joh', 'binnen', 'gekregen']


Top words for topic #6:
['pakket', 'wachten', 'dpd', 'versturen', 'pakketdienst', 'helaas', 'waardoor', 'minimaal', 'zeuren', 'werken']


Top words for topic #7:
['informeren', 'effort', 'nadat'

In [8]:
topic_values = nmf.transform(doc_term_matrix)
dataset['Topic'] = topic_values.argmax(axis=1)
dataset.head(3)

,Headline,Rating,Review,ReviewDate,Topic
0,LET OP: BESTEL HIER NIETS,1,Ruilen en retourneren is moeilijk.\nSlecht con...,2020-01-16T17:09:51Z,3
1,Waardeloze webwinkel,1,25 december een bestelling geplaatst. Het zou ...,2020-01-15T15:15:35Z,5
2,Prima winkel,5,Prima winkel! Veel keus.,2019-12-19T22:30:09Z,9


In [9]:
dataset.Topic = dataset.Topic.astype('str')

dataset.Topic = dataset.Topic.replace(['0'],'Fast delivery / good articles')
dataset.Topic = dataset.Topic.replace(['10'],'Fast delivery / good prices')
dataset.Topic = dataset.Topic.replace(['2'],'Good prices')
dataset.Topic = dataset.Topic.replace(['9'],'Good range of articles / good prices')
dataset.Topic = dataset.Topic.replace(['3'],'Bad service')
dataset.Topic = dataset.Topic.replace(['4'],'Slow delivery')
dataset.Topic = dataset.Topic.replace(['5'],'Bad service / Slow delivery')
dataset.Topic = dataset.Topic.replace(['6'],'Slow delivery / Bad DPD service')
dataset.Topic = dataset.Topic.replace(['7'],'Wrong delivery')
dataset.Topic = dataset.Topic.replace(['14'],'Bad return delivery')

dataset

,Headline,Rating,Review,ReviewDate,Topic
0,LET OP: BESTEL HIER NIETS,1,Ruilen en retourneren is moeilijk.\nSlecht con...,2020-01-16T17:09:51Z,Bad service
1,Waardeloze webwinkel,1,25 december een bestelling geplaatst. Het zou ...,2020-01-15T15:15:35Z,Bad service / Slow delivery
2,Prima winkel,5,Prima winkel! Veel keus.,2019-12-19T22:30:09Z,Good range of articles / good prices
3,zeer negatief! Niet bestellen hier,1,Ook hier een zeer negatieve ervaring. De Rambo...,2019-12-11T15:42:43Z,1
4,Waardeloos,1,Vest bestel in maat S. Bij levering bleek het ...,2019-12-11T14:53:50Z,Bad return delivery
...,...,...,...,...,...
335,Geen woorden voor,1,Als eerste werden mijn chaps kapot geleverd. D...,2019-08-13T14:33:57Z,Bad service
336,Absoluut niet bestellen bij Agradi,1,Absoluut niet bestellen bij dit bedrijf. Ze ho...,2019-08-08T14:28:54Z,Slow delivery
337,Geweldige site gewoon!,5,Geweldige site gewoon!,2019-07-31T11:07:27Z,Good prices
338,Wow!,5,Wow! Fantastisch,2019-07-18T14:38:25Z,8


In [10]:
#dataset.to_csv("Agradi_reviews_with_topics.csv", index=False)

# Topic visualisations

In [11]:
!pip install pyLDAvis
!pip install dill

import pyLDAvis
import pyLDAvis.sklearn
import dill
import warnings

warnings.filterwarnings('ignore')
pyLDAvis.enable_notebook()

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [12]:
top_terms = 5
TOTAL_TOPICS = 15
vocabulary = np.array(tfidf_vect.get_feature_names())
topic_terms = nmf.components_
topic_key_term_idxs = np.argsort(-np.absolute(topic_terms), axis=1)[:, :top_terms]
topic_keyterms = vocabulary[topic_key_term_idxs]
topics = [', '.join(topic) for topic in topic_keyterms]
pd.set_option('display.max_colwidth', -1)
topics_df = pd.DataFrame(topics,
                         columns = ['Terms per Topic'],
                         index=['Topic'+str(t) for t in range(1, TOTAL_TOPICS+1)])
topics_df

,Terms per Topic
Topic1,"snelle, service, levering, super, goede"
Topic2,"deken, twee, wanneer, maand, foto"
Topic3,"site, geweldige, betaalbaar, leuke, gewoon"
Topic4,"slecht, oplichting, contact, ruilen, steeds"
Topic5,"artikel, geleverd, bedrijf, maand, zwaar"
Topic6,"gekregen, binnen, december, vertraging, excusses"
Topic7,"helaas, waardoor, pakketdienst, zeuren, versturen"
Topic8,"goedkoop, duidelijk, bedrijf, effort, mis"
Topic9,"fantastisch, wow, foto, gemaild, opnieuw"
Topic10,"prima, keus, winkel, aangemaakt, printer"


In [13]:
#next step is to rename the topic column

topics_df['Topic2'] = topics_df.index
topics_df.Topic2.astype('str')

topics_df.Topic2 = topics_df.Topic2.replace(['Topic1'],'Fast delivery / good articles')
topics_df.Topic2 = topics_df.Topic2.replace(['Topic11'],'Fast delivery / good prices')
topics_df.Topic2 = topics_df.Topic2.replace(['Topic3'],'Good prices')
topics_df.Topic2 = topics_df.Topic2.replace(['Topic10'],'Good range of articles / good prices')
topics_df.Topic2 = topics_df.Topic2.replace(['Topic4'],'Bad service')
topics_df.Topic2 = topics_df.Topic2.replace(['Topic5'],'Slow delivery')
topics_df.Topic2 = topics_df.Topic2.replace(['Topic6'],'Bad service / Slow delivery')
topics_df.Topic2 = topics_df.Topic2.replace(['Topic7'],'Slow delivery / Bad DPD service')
topics_df.Topic2 = topics_df.Topic2.replace(['Topic8'],'Wrong delivery')
topics_df.Topic2 = topics_df.Topic2.replace(['Topic15'],'Bad return delivery')
topics_df

,Terms per Topic,Topic2
Topic1,"snelle, service, levering, super, goede",Fast delivery / good articles
Topic2,"deken, twee, wanneer, maand, foto",Topic2
Topic3,"site, geweldige, betaalbaar, leuke, gewoon",Good prices
Topic4,"slecht, oplichting, contact, ruilen, steeds",Bad service
Topic5,"artikel, geleverd, bedrijf, maand, zwaar",Slow delivery
Topic6,"gekregen, binnen, december, vertraging, excusses",Bad service / Slow delivery
Topic7,"helaas, waardoor, pakketdienst, zeuren, versturen",Slow delivery / Bad DPD service
Topic8,"goedkoop, duidelijk, bedrijf, effort, mis",Wrong delivery
Topic9,"fantastisch, wow, foto, gemaild, opnieuw",Topic9
Topic10,"prima, keus, winkel, aangemaakt, printer",Good range of articles / good prices


In [16]:
# Now JOIN with previous df

topics_df2 = topics_df.copy()
topics_df2 
df4 = pd.merge(dataset, topics_df2, left_on='Topic', right_on='Topic2', how='left').drop('Topic2', axis=1)
df4 ['CountOfRankings'] = df4.groupby('Rating')['Rating'].transform('count')
df4 ['CountOfTopics'] = df4.groupby('Topic')['Topic'].transform('count')
df4.head(5)


,Headline,Rating,Review,ReviewDate,Topic,Terms per Topic,CountOfRankings,CountOfTopics
0,LET OP: BESTEL HIER NIETS,1,Ruilen en retourneren is moeilijk.\nSlecht contact en nog steeds geen geld terug.\nGewoon niks bestellen hier.\nOplichting!,2020-01-16T17:09:51Z,Bad service,"slecht, oplichting, contact, ruilen, steeds",170,34
1,Waardeloze webwinkel,1,25 december een bestelling geplaatst. Het zou binnen 1 tot 4 werkdagen binnen komen. Het is nu 15 januari en vandaag pas een mail gekregen dat het pakket vertraging heeft?? Joh.. ik heb gebeld en gezegd dat ik mijn geld terug wil. Geen excusses niks gekregen. NOOIT MEER AGRADI VOOR MIJ!,2020-01-15T15:15:35Z,Bad service / Slow delivery,"gekregen, binnen, december, vertraging, excusses",170,17
2,Prima winkel,5,Prima winkel! Veel keus.,2019-12-19T22:30:09Z,Good range of articles / good prices,"prima, keus, winkel, aangemaakt, printer",136,17
3,zeer negatief! Niet bestellen hier,1,"Ook hier een zeer negatieve ervaring. De Rambo duo combo besteld die dus bestaat uit drie dekens, waren er maar twee geleverd. Kan gebeuren zou je denk maar wanneer je belt word je aan het lijntje gehouden en dit voor een maand lang. Ze blijven vaag in hun antwoorden en geven nooit geen inhoudelijke reactie over wanneer ik de deken zou ontvangen. Na een maand was ik het wel beu en heb ik ze in gebreke gesteld en voorgesteld om maar met rechtsbijstand verder te gaan. En jawel het wonder is geschied binnen twee dagen mijn deken binnen!!! Mensen niet bestellen bij Agradi het gaat daar niet goed",2019-12-11T15:42:43Z,1,NaN,170,34
4,Waardeloos,1,"Vest bestel in maat S. Bij levering bleek het de verkeerde maat te zijn. Jammer, maar kan gebeuren. Maar dan wil je de klantenservice bellen. Eerst eindeloos wachten, dan word je eruit gegooid. Na een halve dag proberen, eindelijk iemand aan de telefoon. Het vest kan geretourneerd worden en ze sturen de goede maat zodra de retour verwerkt is. Best raar want het was hun fout. Ik nog drie keer nagevraagd of het vest dan zeker nog op voorraad zou zijn, maar dat was geen probleem. Vandaag gekeken of mijn retour eindelijk al eens verwerkt was, zie ik dat het vest uitverkocht is. Weer gebeld, nee sorry die hebben we niet meer. Krijg ook mijn geld nog niet terug want de retourzending is nog niet binnen. Echt belachelijke service.",2019-12-11T14:53:50Z,Bad return delivery,"vest, maat, eindelijk, verwerkt, retour",170,17


In [17]:
df4.to_csv("Agradi_Sentiment.csv", index=False)